In [1]:
import pandas as pd
import altair as alt
import numpy as np

from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('SeekUT_TexasData_2020.xlsx', engine='openpyxl')

df.head()

,inst_name,degree_level,CIP2,CIP2Title,CIP6,CIP6Title,STEMStatus,CIP6Definition,Cohort_CNT1,NMatched_Y1,...,DTI_aggr_Ratio_YR10,DTI_Loan_Monthly_Median_Yr1,DTI_Loan_Monthly_Median_Yr5,DTI_Loan_Monthly_Median_Yr10,DTI_Year1wage_monthly_Median,DTI_Year5wage_monthly_Median,DTI_Year10wage_monthly_Median,firstyr_resi_tuitionfee,popular_maj_count,Loan_PCT
0,UT Arlington,Baccalaureate,3,Natural Resources And Conservation,3.0104,Environmental Science,STEM,A program that focuses on the application of b...,25.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.556
1,UT Arlington,Baccalaureate,4,Architecture And Related Services,4.0201,Architecture,NON-STEM,A program that prepares individuals for the in...,197.0,111.0,...,2.3,276.256079,281.045871,145.45967,3540.341800,4987.185894,6232.302830,NaN,54.0,0.611
2,UT Arlington,Baccalaureate,4,Architecture And Related Services,4.0501,Interior Architecture,NON-STEM,A program that prepares individuals to apply a...,55.0,39.0,...,1.6,250.059581,202.610653,89.68415,3768.877887,4857.173009,5467.573651,NaN,7.0,0.364
3,UT Arlington,Baccalaureate,11,Computer And Information Sciences And Support ...,11.0401,Information Science/Studies,STEM,"A program that focuses on the theory, organiz...",531.0,384.0,...,NaN,275.302708,324.509701,NaN,4395.408420,6669.958333,NaN,NaN,189.0,0.523
4,UT Arlington,Baccalaureate,11,Computer And Information Sciences And Support ...,11.0701,Computer Science,STEM,"A program that focuses on computer theory, com...",294.0,203.0,...,NaN,216.302814,240.296427,NaN,5984.857055,7543.691223,NaN,NaN,121.0,0.407


In [2]:
df = df[ df['degree_level'] == 'Baccalaureate']

In [3]:
df_yr1 = df.groupby('inst_name')['Year1Wage_Median'].median()
df_yr5 = df.groupby('inst_name')['Year5Wage_Median'].median()
df_yr10 = df.groupby('inst_name')['Year10Wage_Median'].median()

In [4]:
summary_df = pd.concat([df_yr1, df_yr5, df_yr10], axis= 1)
summary_df = summary_df.reset_index()

In [5]:
# use df_median for the 1st visualization (line graph)
df_median = df[['inst_name', 'CIP6Title', 'CIP2Title',
                'Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median']]

df_median.head()

,inst_name,CIP6Title,CIP2Title,Year1Wage_Median,Year5Wage_Median,Year10Wage_Median
0,UT Arlington,Environmental Science,Natural Resources And Conservation,37991.0,NaN,NaN
1,UT Arlington,Architecture,Architecture And Related Services,42205.0,60935.0,74148.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,39525.0,53368.0,62107.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,54515.0,75717.0,NaN
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,72212.0,91598.0,104794.0


In [6]:
# add new records for each institution for each year's median values (summary_df) to df_median
df_median = df_median.append(summary_df)
df_median

,inst_name,CIP6Title,CIP2Title,Year1Wage_Median,Year5Wage_Median,Year10Wage_Median
0,UT Arlington,Environmental Science,Natural Resources And Conservation,37991.0,NaN,NaN
1,UT Arlington,Architecture,Architecture And Related Services,42205.0,60935.0,74148.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,39525.0,53368.0,62107.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,54515.0,75717.0,NaN
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,72212.0,91598.0,104794.0
...,...,...,...,...,...,...
8,UT Permian Basin,NaN,NaN,48747.5,55339.0,62192.5
9,UT Rio Grande Valley,NaN,NaN,32839.0,50159.0,55327.0
10,UT San Antonio,NaN,NaN,37372.0,54082.0,60897.0
11,UT Southwestern Medical Center,NaN,NaN,NaN,77977.0,NaN


Note that the rows with CIP6Title as NaN are the summary ones with the median earnings for each institution.

In [7]:
df_median = pd.melt(df_median, id_vars=['inst_name', 'CIP6Title', 'CIP2Title'], 
                    value_vars=['Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median'])

In [8]:
df_median.head()

,inst_name,CIP6Title,CIP2Title,variable,value
0,UT Arlington,Environmental Science,Natural Resources And Conservation,Year1Wage_Median,37991.0
1,UT Arlington,Architecture,Architecture And Related Services,Year1Wage_Median,42205.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,Year1Wage_Median,39525.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,Year1Wage_Median,54515.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,Year1Wage_Median,72212.0


In [9]:
df_median = df_median.reset_index(drop=True)

In [10]:
# convert year 1 wage to just a number of 1... make it continuous
for i in range(0, len(df_median)):
    if df_median.loc[i, 'variable'] == 'Year1Wage_Median':
        df_median.loc[i, 'variable'] = 1
    elif df_median.loc[i, 'variable'] == 'Year5Wage_Median':
        df_median.loc[i, 'variable'] = 5
    else:
        df_median.loc[i, 'variable'] = 10
    
df_median

,inst_name,CIP6Title,CIP2Title,variable,value
0,UT Arlington,Environmental Science,Natural Resources And Conservation,1,37991.0
1,UT Arlington,Architecture,Architecture And Related Services,1,42205.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,1,39525.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,1,54515.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,1,72212.0
...,...,...,...,...,...
1471,UT Permian Basin,NaN,NaN,10,62192.5
1472,UT Rio Grande Valley,NaN,NaN,10,55327.0
1473,UT San Antonio,NaN,NaN,10,60897.0
1474,UT Southwestern Medical Center,NaN,NaN,10,NaN


In [11]:
df_median['variable'].value_counts()

1     492
5     492
10    492
Name: variable, dtype: int64

In [12]:
# rename columns for legibility for user
df_median.rename(columns = {'CIP6Title' : 'Major', 
                            'value': 'Salary', 
                            'CIP2Title' : 'Industry',
                            'variable' : 'Year'
                           }, inplace = True)

# replace NA values for 'Major' and 'Industry' with 'Institution Median' based on calculation above
df_median['Major'].fillna("Institution Median", inplace = True)
df_median['Industry'].fillna("Institution Median", inplace = True)

In [13]:
df_median

,inst_name,Major,Industry,Year,Salary
0,UT Arlington,Environmental Science,Natural Resources And Conservation,1,37991.0
1,UT Arlington,Architecture,Architecture And Related Services,1,42205.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,1,39525.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,1,54515.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,1,72212.0
...,...,...,...,...,...
1471,UT Permian Basin,Institution Median,Institution Median,10,62192.5
1472,UT Rio Grande Valley,Institution Median,Institution Median,10,55327.0
1473,UT San Antonio,Institution Median,Institution Median,10,60897.0
1474,UT Southwestern Medical Center,Institution Median,Institution Median,10,NaN


In [14]:
for i in range(0, len(df_median)):
    if np.isnan(df_median.loc[i, 'Salary']) == False:
        df_median.loc[i, 'f_Salary'] = "$" + str(df_median.loc[i, 'Salary'])
    
df_median

,inst_name,Major,Industry,Year,Salary,f_Salary
0,UT Arlington,Environmental Science,Natural Resources And Conservation,1,37991.0,$37991.0
1,UT Arlington,Architecture,Architecture And Related Services,1,42205.0,$42205.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,1,39525.0,$39525.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,1,54515.0,$54515.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,1,72212.0,$72212.0
...,...,...,...,...,...,...
1471,UT Permian Basin,Institution Median,Institution Median,10,62192.5,$62192.5
1472,UT Rio Grande Valley,Institution Median,Institution Median,10,55327.0,$55327.0
1473,UT San Antonio,Institution Median,Institution Median,10,60897.0,$60897.0
1474,UT Southwestern Medical Center,Institution Median,Institution Median,10,NaN,NaN


In [15]:
# edit legend to only show selected items?
# or don't show legend at all since we can use tooltip

# list of unique majors
industries = df_median['Industry'].unique()

# list of institutions
institutions = df_median['inst_name'].unique()

# list of years from graduation
years = df_median['Year'].unique()

# used for study_selector (selecting industries) 
# removed :O categorization for column name
industry_df = pd.DataFrame({'Industry': industries})

# select institution dropdown
inst_dropdown = alt.binding_select(options=institutions)
inst_select = alt.selection_single(
    fields=['inst_name'], 
    bind=inst_dropdown, 
    name="Institution")

# checkboxes to select industry
selection = alt.selection_multi(fields=['Industry'])
color = alt.condition(selection, alt.Color('Industry:N'), alt.value('lightgray'))
study_selector = alt.Chart(industry_df).mark_rect(tooltip = True).encode(
    y = alt.Y('Industry:N', title = ""), 
    color = color,
    tooltip = ['Industry']
).add_selection(selection)

# want the list of industries to change dynamically depending on selected institution
salary_info = alt.Chart(df_median).mark_line(point = True, tooltip = True).encode(
    x= alt.X('Year:Q', title = "Years from Graduation", axis=alt.Axis(tickMinStep=1)),
    y= alt.Y('mean(Salary):Q', scale = alt.Scale(round = True)), 
    color= alt.Color('Industry:N', legend = None),
    tooltip = ['Industry', 'Year', 'mean(Salary)']
).properties(
    width = 500,
    title = 'Mean Salaries After Graduation by Industry'
).add_selection(
    inst_select
).transform_filter(
    inst_select
).transform_filter(
    selection
).interactive()

study_selector | salary_info

alt.HConcatChart(...)

# Visualization 2


In [16]:
# gather necessary variables for boxplot and histogram
df_industry = df[['inst_name', 'CIP6Title','CIP2Title', 'STEMStatus',
                'Year1Wage_Mean', 'Year1Wage_Median', 'Year1Wage_Min', 'Year1Wage_Max', 
                  'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75', 'Year1Wage_P90',
                  'Year5Wage_Mean', 'Year5Wage_Median', 'Year5Wage_Max' , 'Year5Wage_Min' , 
                  'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75', 'Year5Wage_P90', 
                  'Year10Wage_Mean' , 'Year10Wage_Median' , 'Year10Wage_Max' ,'Year10Wage_Min' ,
                  'Year10Wage_P10' , 'Year10Wage_P25' , 'Year10Wage_P75' , 'Year10Wage_P90']]

In [17]:
# melt dataframe from wide to long 
df_industry = pd.melt(df_industry, 
               id_vars=['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus'], 
               value_vars=['Year1Wage_Mean', 'Year1Wage_Median', 'Year1Wage_Min', 'Year1Wage_Max', 
                           'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75', 'Year1Wage_P90',
                           'Year5Wage_Mean', 'Year5Wage_Median', 'Year5Wage_Max' , 'Year5Wage_Min' , 
                           'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75', 'Year5Wage_P90', 
                           'Year10Wage_Mean' , 'Year10Wage_Median' , 'Year10Wage_Max' ,'Year10Wage_Min' ,
                           'Year10Wage_P10' , 'Year10Wage_P25' , 'Year10Wage_P75' , 'Year10Wage_P90'])

df_industry

,inst_name,CIP6Title,CIP2Title,STEMStatus,variable,value
0,UT Arlington,Environmental Science,Natural Resources And Conservation,STEM,Year1Wage_Mean,37312.0
1,UT Arlington,Architecture,Architecture And Related Services,NON-STEM,Year1Wage_Mean,42191.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,NON-STEM,Year1Wage_Mean,42462.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,STEM,Year1Wage_Mean,56053.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,STEM,Year1Wage_Mean,68837.0
...,...,...,...,...,...,...
11491,UT Tyler,"Finance, General","Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,143873.0
11492,UT Tyler,Human Resources Development,"Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,97019.0
11493,UT Tyler,"Marketing/Marketing Management, General","Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,148657.0
11494,UT Tyler,Construction Management,"Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,NaN


In [18]:
# convert year 1 wage to just a number of 1... make it continuous
for i in range(0, len(df_industry)):
    if df_industry.loc[i, 'variable'].startswith('Year1Wage'):
        df_industry.loc[i, 'Year'] = 1
    elif df_industry.loc[i, 'variable'].startswith('Year5Wage'):
        df_industry.loc[i, 'Year'] = 5
    else:
        df_industry.loc[i, 'Year'] = 10
    
df_industry

,inst_name,CIP6Title,CIP2Title,STEMStatus,variable,value,Year
0,UT Arlington,Environmental Science,Natural Resources And Conservation,STEM,Year1Wage_Mean,37312.0,1.0
1,UT Arlington,Architecture,Architecture And Related Services,NON-STEM,Year1Wage_Mean,42191.0,1.0
2,UT Arlington,Interior Architecture,Architecture And Related Services,NON-STEM,Year1Wage_Mean,42462.0,1.0
3,UT Arlington,Information Science/Studies,Computer And Information Sciences And Support ...,STEM,Year1Wage_Mean,56053.0,1.0
4,UT Arlington,Computer Science,Computer And Information Sciences And Support ...,STEM,Year1Wage_Mean,68837.0,1.0
...,...,...,...,...,...,...,...
11491,UT Tyler,"Finance, General","Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,143873.0,10.0
11492,UT Tyler,Human Resources Development,"Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,97019.0,10.0
11493,UT Tyler,"Marketing/Marketing Management, General","Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,148657.0,10.0
11494,UT Tyler,Construction Management,"Business, Management, Marketing, And Related S...",NON-STEM,Year10Wage_P90,NaN,10.0


In [19]:
# pivot so that year is in a separate column to enable radio button functionality
df_industry = df_industry.pivot(index = ['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus', 'Year'], 
                      columns = 'variable', 
                      values = 'value')

In [20]:
# reset index and restore columns that were used for the index for use in the visualization
df_industry.reset_index(['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus', 'Year'], inplace = True)
df_industry

variable,inst_name,CIP6Title,CIP2Title,STEMStatus,Year,Year10Wage_Max,Year10Wage_Mean,Year10Wage_Median,Year10Wage_Min,Year10Wage_P10,...,Year1Wage_P75,Year1Wage_P90,Year5Wage_Max,Year5Wage_Mean,Year5Wage_Median,Year5Wage_Min,Year5Wage_P10,Year5Wage_P25,Year5Wage_P75,Year5Wage_P90
0,UT Arlington,Accounting,"Business, Management, Marketing, And Related S...",NON-STEM,1.0,NaN,NaN,NaN,NaN,NaN,...,59194.0,67766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UT Arlington,Accounting,"Business, Management, Marketing, And Related S...",NON-STEM,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,219817.0,70074.0,68225.0,13569.0,42460.0,53459.0,83022.0,95751.0
2,UT Arlington,Accounting,"Business, Management, Marketing, And Related S...",NON-STEM,10.0,415601.0,90354.0,84543.0,21624.0,49182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UT Arlington,"Aerospace, Aeronautical and Astronautical/Spac...",Engineering,STEM,1.0,NaN,NaN,NaN,NaN,NaN,...,68960.0,76760.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UT Arlington,"Aerospace, Aeronautical and Astronautical/Spac...",Engineering,STEM,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,145701.0,78519.0,77926.0,15178.0,56538.0,70000.0,90205.0,105067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,UT Tyler,"Social Sciences, General",Social Sciences,NON-STEM,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1433,UT Tyler,"Social Sciences, General",Social Sciences,NON-STEM,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1434,UT Tyler,Spanish Language and Literature,"Foreign Languages, Literatures, And Linguistics",NON-STEM,1.0,NaN,NaN,NaN,NaN,NaN,...,39825.0,52982.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1435,UT Tyler,Spanish Language and Literature,"Foreign Languages, Literatures, And Linguistics",NON-STEM,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,135495.0,49061.0,44775.0,15158.0,23774.0,37876.0,56140.0,71744.0


In [21]:
df_industry.iloc[100]

variable
inst_name                                                 UT Arlington
CIP6Title                        International Business/Trade/Commerce
CIP2Title            Business, Management, Marketing, And Related S...
STEMStatus                                                    NON-STEM
Year                                                               5.0
Year10Wage_Max                                                     NaN
Year10Wage_Mean                                                    NaN
Year10Wage_Median                                                  NaN
Year10Wage_Min                                                     NaN
Year10Wage_P10                                                     NaN
Year10Wage_P25                                                     NaN
Year10Wage_P75                                                     NaN
Year10Wage_P90                                                     NaN
Year1Wage_Max                                                      N

In [22]:
df_industry.columns

Index(['inst_name', 'CIP6Title', 'CIP2Title', 'STEMStatus', 'Year',
       'Year10Wage_Max', 'Year10Wage_Mean', 'Year10Wage_Median',
       'Year10Wage_Min', 'Year10Wage_P10', 'Year10Wage_P25', 'Year10Wage_P75',
       'Year10Wage_P90', 'Year1Wage_Max', 'Year1Wage_Mean', 'Year1Wage_Median',
       'Year1Wage_Min', 'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75',
       'Year1Wage_P90', 'Year5Wage_Max', 'Year5Wage_Mean', 'Year5Wage_Median',
       'Year5Wage_Min', 'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75',
       'Year5Wage_P90'],
      dtype='object', name='variable')

In [33]:
val_cols = ['Year10Wage_Max', 'Year10Wage_Mean', 'Year10Wage_Median',
       'Year10Wage_Min', 'Year10Wage_P10', 'Year10Wage_P25', 'Year10Wage_P75',
       'Year10Wage_P90', 'Year1Wage_Max', 'Year1Wage_Mean', 'Year1Wage_Median',
       'Year1Wage_Min', 'Year1Wage_P10', 'Year1Wage_P25', 'Year1Wage_P75',
       'Year1Wage_P90', 'Year5Wage_Max', 'Year5Wage_Mean', 'Year5Wage_Median',
       'Year5Wage_Min', 'Year5Wage_P10', 'Year5Wage_P25', 'Year5Wage_P75',
       'Year5Wage_P90']

# rename columns to eliminate Year classifier in each name - allow radio buttons to filter by Year
max_cols = ['Year1Wage_Max', 'Year5Wage_Max', 'Year10Wage_Max']
mean_cols = ['Year1Wage_Mean', 'Year5Wage_Mean', 'Year10Wage_Mean']
median_cols = ['Year1Wage_Median', 'Year5Wage_Median', 'Year10Wage_Median']
min_cols = ['Year1Wage_Min', 'Year5Wage_Min', 'Year10Wage_Min']

p10_cols = ['Year1Wage_P10', 'Year5Wage_P10', 'Year10Wage_P10']
p25_cols = ['Year1Wage_P25', 'Year5Wage_P25', 'Year10Wage_P25']
p75_cols = ['Year1Wage_P75', 'Year5Wage_P75', 'Year10Wage_P75']
p90_cols = ['Year1Wage_P90', 'Year5Wage_P90', 'Year10Wage_P90']


df_industry['max'] = pd.to_numeric(df_industry[max_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['mean'] = pd.to_numeric(df_industry[mean_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['median'] = pd.to_numeric(df_industry[median_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['min'] = pd.to_numeric(df_industry[min_cols].bfill(axis=1).iloc[:, 0], errors='coerce')

df_industry['p10'] = pd.to_numeric(df_industry[p10_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p25'] = pd.to_numeric(df_industry[p25_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p75'] = pd.to_numeric(df_industry[p75_cols].bfill(axis=1).iloc[:, 0], errors='coerce')
df_industry['p90'] = pd.to_numeric(df_industry[p90_cols].bfill(axis=1).iloc[:, 0], errors='coerce')

In [42]:
df_industry.iloc[410]

variable
inst_name                               UT Austin
CIP6Title            Music Theory and Composition
CIP2Title              Visual And Performing Arts
STEMStatus                               NON-STEM
Year                                         10.0
Year10Wage_Max                           109631.0
Year10Wage_Mean                           66225.0
Year10Wage_Median                         56291.0
Year10Wage_Min                            37172.0
Year10Wage_P10                            37172.0
Year10Wage_P25                            50972.0
Year10Wage_P75                            84236.0
Year10Wage_P90                           109631.0
Year1Wage_Max                                 NaN
Year1Wage_Mean                                NaN
Year1Wage_Median                              NaN
Year1Wage_Min                                 NaN
Year1Wage_P10                                 NaN
Year1Wage_P25                                 NaN
Year1Wage_P75                            

In [41]:
alt.data_transformers.disable_max_rows()

# establish the color mapping
scale = alt.Scale(domain=['STEM', 'NON-STEM'],
                  range=['#5ab4ac', '#d8b365'])

# radio buttons to select year
years = df_industry['Year'].unique()
year_radio = alt.binding_radio(options=years)
year_select = alt.selection_single(
    fields=['Year'], 
    bind=year_radio, 
    name="Year",
    init = {'Year': 5})

year_color_condition = alt.condition(year_select,
                      alt.Color('Year:Q', legend=None),
                      alt.value('lightgray'))

#list of institutions
institutionss = df_industry['inst_name'].unique()

# select institution dropdown
inst_dropdowns = alt.binding_select(options=institutionss)
inst_selects = alt.selection_single(
    fields=['inst_name'], 
    bind=inst_dropdowns, 
    name="inst_name",
    init = {'inst_name' : "UT Austin"}
)

interval = alt.selection_multi(on='click', encodings=['y'])

chart = alt.Chart(df_industry).mark_bar().encode(
    y = alt.Y('CIP2Title:N', axis=alt.Axis(title='Industry')),
    x = alt.X('mean(median):Q', axis=alt.Axis(title='Salary')),
    color = alt.condition(interval, 'STEMStatus:N', alt.value('lightgray'), scale=scale),
    tooltip = ['CIP2Title:N','mean(median):Q']
).properties(
    title = "Salary Distribution by Industry",
).add_selection(
    interval,
    inst_selects,
    year_select
).transform_filter(
    inst_selects
).transform_filter(
    year_select
).interactive()

# box = alt.LayerChart(df_ind_austin).transform_aggregate(
# min="Year1Wage_Min:Q",
# max="Year1Wage_Max:Q",
# mean="Year1Wage_Mean:Q",
# q1="Year1Wage_P25:Q",
# q3="Year1Wage_P75:Q"

box = alt.LayerChart(df_industry).encode(
tooltip = ['inst_name:N', 'CIP6Title:N','min:Q', 'p10:Q', 'p25:Q', 'median:Q', 'p75:Q', 'p90:Q', 'max:Q'],
y = alt.Y('CIP6Title:N', title = 'Majors')
).add_layers(
alt.Chart().mark_rule(tooltip=True).encode(x=alt.X('median(p10):Q', title='Salary in Dollars ($)'), x2='min(p90):Q').interactive(),
alt.Chart().mark_bar(size = 15, height = 20, tooltip=True).encode(x='mean(p25):Q', x2='mean(p75):Q', color=alt.Color('STEMStatus:N')).interactive(),
alt.Chart().mark_tick(color='white', height=18,tooltip=True).encode(x='mean(median):Q').interactive()
).transform_filter(
    interval
).transform_filter(
    inst_selects
).properties(
    title='Distribution of Salaries of Majors for Above Selection'
).interactive()

chart & box.add_selection(alt.selection_single())

alt.VConcatChart(...)

In [25]:
# alt.data_transformers.disable_max_rows()

# # establish the color mapping
# scale = alt.Scale(domain=['STEM', 'NON-STEM'],
#                   range=['#5ab4ac', '#d8b365'])

# # radio buttons to select year
# years = df_industry['Year'].unique()
# year_radio = alt.binding_radio(options=years)
# year_select = alt.selection_single(fields=['Year'], bind=year_radio, name="Year")
# year_color_condition = alt.condition(year_select,
#                       alt.Color('Year:Q', legend=None),
#                       alt.value('lightgray'))

# #list of institutions
# institutionss = df_industry['inst_name'].unique()

# # select institution dropdown
# inst_dropdowns = alt.binding_select(options=institutionss)
# inst_selects = alt.selection_single(
#     fields=['inst_name'], 
#     bind=inst_dropdowns, 
#     name="inst_name",
#     init = {'inst_name' : "UT Austin"}
# )

# interval = alt.selection_multi(on='click', encodings=['y'])

# chart = alt.Chart(df_industry).mark_bar().encode(
#     y = alt.Y('CIP2Title:N', axis=alt.Axis(title='Industry')),
#     x = alt.X('mean(Year5Wage_Median):Q', axis=alt.Axis(title='Salary')),
#     color = alt.condition(interval, 'STEMStatus:N', alt.value('lightgray'), scale=scale),
#     tooltip = ['mean(Year5Wage_Median):Q','CIP2Title:N']
# ).properties(
#     title = "Salary Distribution by Industry",
# ).add_selection(
#     interval,
#     inst_selects,
#     year_select
# ).transform_filter(
#     inst_selects
# ).transform_filter(
#     year_select
# ).interactive()

# # box = alt.LayerChart(df_ind_austin).transform_aggregate(
# # min="Year1Wage_Min:Q",
# # max="Year1Wage_Max:Q",
# # mean="Year1Wage_Mean:Q",
# # q1="Year1Wage_P25:Q",
# # q3="Year1Wage_P75:Q"

# box = alt.LayerChart(df_industry).encode(
# tooltip = ['inst_name:N', 'CIP6Title:N','Year1Wage_Min:Q','Year1Wage_P25:Q', 'Year1Wage_Mean:Q', 'Year1Wage_P75:Q', 'Year1Wage_Max:Q'],
# y= alt.Y('CIP6Title:N', title = 'Majors')
# ).add_layers(
# alt.Chart().mark_rule(tooltip=True).encode(x=alt.X('median(Year1Wage_P10):Q', title='Salary in Dollars ($)'), x2='min(Year1Wage_P90):Q').interactive(),
# alt.Chart().mark_bar(size = 15, height = 20, tooltip=True).encode(x='mean(Year1Wage_P25):Q', x2='mean(Year1Wage_P75):Q', color=alt.Color('STEMStatus:N')).interactive(),
# alt.Chart().mark_tick(color='white', height=18,tooltip=True).encode(x='mean(Year1Wage_Median):Q').interactive()
# ).transform_filter(
#     interval
# ).transform_filter(
#     inst_selects
# ).properties(
#     title='Distribution of Salaries of Majors for Above Selection'
# ).interactive()

# chart & box.add_selection(alt.selection_single())

alt.VConcatChart(...)

In [26]:
# old code visualization 
# df_austin['CIP6Title'].fillna("Institution Median", inplace = True)
# majors = df_austin['CIP6Title'].unique()
# #how to replace NA with school median

# major_dropdown = alt.binding_select(options=majors)
# # major_select = alt.selection_single(fields=['CIP6Title'], bind=major_dropdown, name="Major")


# #use this to get unique list
# # df_austin = df_median [ df_median['inst_name'] == 'UT Austin' ]

# #use df austin


# #add dropdown list

# # line = alt.Chart(df_austin).mark_line(point = True).encode(
# #     x='variable:Q',
# #     y='value:Q',
# # #     color='CIP6Title:N'
# #     color = alt.condition(major_select, alt.Color('CIP6Title:N'), alt.value('lightgray')),
# #     tooltip=['value:Q', 'CIP6Title:O']
# # ).properties(width = 500).add_selection(
# #     major_select
# #  ).transform_filter(
# #      major_select
# # ).properties(title="Salary by Major at UT Austin")

# # line

# #removed :O categorization
# majorsDF = pd.DataFrame({'CIP6Title': majors})

# #institution selector

# majors = df_austin['CIP6Title'].unique()
# #how to replace NA with school median

# institutions = df_austin['inst_name'].unique()
# inst_dropdown = alt.binding_select(options=institutions)
# inst_select = alt.selection_single(fields=['inst_name'], bind=inst_dropdown, name="Institution")


# selection = alt.selection_multi(fields=['CIP6Title'])
# color = alt.condition(selection, alt.Color('CIP6Title:N'), alt.value('lightgray'))
# #make replaced with CIP6title
# study_selector = alt.Chart(majorsDF).mark_rect().encode(y='CIP6Title', color=color).add_selection(selection)

# #fuel replaced with value
# salary_info = alt.Chart(df_austin).mark_line().encode(x='variable:Q', y=alt.Y('value:Q'), color='CIP6Title:N').properties(width = 500).add_selection(
#     inst_select).transform_filter(selection).transform_filter()


# study_selector | salary_info


# #attempting to add check boxes

# #current code just results in bigger lines until check box, does not actually check against school

# # input_checkbox = alt.binding_checkbox()
# # checkbox_selection = alt.selection_single(bind=input_checkbox, name="UT Austin")

# # size_checkbox_condition = alt.condition(checkbox_selection,
# #                                         alt.SizeValue(25),
# #                                         alt.Size('inst_name:O')
# #                                        )

# # school_check = line.add_selection(
# #     checkbox_selection
# # ).encode(
# #     size=size_checkbox_condition
# # ).properties(title="Checkbox Formatting")

# #attempting to add more complicaated trace

# # line

In [27]:
alt.data_transformers.disable_max_rows()
len(df_austin)
df_austin.tail()


interval = alt.selection_multi(on='click', encodings=['y'])
chart = alt.Chart(df_ind_austin).mark_bar().encode(
    y = alt.Y('CIP2Title:O', axis=alt.Axis(title='Industry')),
    x = alt.X('mean(Year5Wage_Median):Q', axis=alt.Axis(title='Salary')),
    color=alt.condition(interval, 'STEMStatus:N', alt.value('lightgray')),
    tooltip = ['mean(Year5Wage_Median):Q','CIP2Title:O']
).properties(
    title = "Salary Distribution by Industry",
).add_selection(
#     select_year,
    interval
)
# .transform_filter(
#     select_year
# )

# df_austin_dist = df_austin['CIP6Title'].unique()
# df_austin_dist.head()


hist = alt.Chart(df_ind_austin).mark_bar().encode(
    x= alt.X('Year5Wage_Median:Q', axis=alt.Axis(title='Salary')),
    y=alt.Y('CIP6Title:O', axis=alt.Axis(title='Major')),
    color=alt.Color('STEMStatus:N')
).transform_filter(
    interval
).properties(
    title='Distribution of Salaries of Majors for Above Selection'
)
chart & hist

#instead of slider, make radio buttons

NameError: name 'df_austin' is not defined